<a href="https://colab.research.google.com/github/souceira/wildfirewatch/blob/dennybranch/WildfireWatchFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all packages
import numpy as np
import pandas as pd
import io
from sklearn import linear_model
from google.colab import files
from datetime import date
from geopy import Nominatim

# Import model data
uploaded = files.upload()
dat = pd.read_csv(io.BytesIO(uploaded['predictivedat.csv']))

Saving predictivedat.csv to predictivedat (1).csv


In [ ]:
# Fit model
x = dat[['lat', 'long', 'apr_temp', 'may_temp', 'june_temp', 'july_temp',  'aug_temp',
        'apr_rain', 'may_rain', 'june_rain', 'july_rain', 'aug_rain']]
y = dat['date_fire']
lm = linear_model.LinearRegression()
WFW_model = lm.fit(x, y)

# Obtain user-inputted info and reject if not for BC
current_prov = input("What province? ")
if current_prov in ('bc', 'BC', 'british columbia', 'British Columbia'):
  print("Great! We can help you predict when wildfires will happen in British Columbia.")
  current_city = input("Which city or region? ")
  current_month = input("What month? (Please enter a number from 1 to 12) ")
  current_day = input("What day? (Please enter a number from 1 to 31) ")
  print("Thanks, we are now predicting risk of wildfires near you.")
else:
  print("Sorry, that province is not supported at this time. Check back soon!")

# Process user-inputted location data
geolocator = Nominatim(user_agent="my_user_agent")
loc = geolocator.geocode(current_city)
lati = loc.longitude
longi = loc.latitude

# Set Kelowna weather parameters for pitch
temp4 = 16
temp5 = 20
temp6 = 27
temp7 = 31
temp8 = 32
rain4 = 3
rain5 = 1
rain6 = 2
rain7 = 3
rain8 = 3

# Compile user's dataframe
user_dat = pd.DataFrame(np.array([[lati, longi, temp4, temp5, temp6, temp7, temp8,
                                   rain4, rain5, rain6, rain7, rain8]]),
                   columns=['lat', 'long', 'apr_temp', 'may_temp', 'june_temp', 'july_temp', 'aug_temp', 
                            'apr_rain', 'may_rain', 'june_rain', 'july_rain', 'aug_rain'])

# Obtain predicted value from model
predicted_val = WFW_model.predict(user_dat)
predicted_val = predicted_val[0]

# Set first day as New Year's Day
first_day = date(2023, 1, 1)

# Process user-inputted data into formatted date
current_date = date(2023, int(current_month), int(current_day)) 

# Get the difference in days - our observed days from first day
diff_days = current_date - first_day
observed_val = diff_days.days

# Get the accuracy in days = difference between predicted vs. observed days from first day
acc_days = round(abs(predicted_val - observed_val), 0)
acc_months = round(acc_days/30, 1)

# Add space and print the universal results text
print("          ")
print("Thanks for waiting! See below for your results.")
print("          ")
print(" ------------------------------- ")
print("|                               |")
print("|   ~  P R E D I C T I O N   ~  |")
print("|         R E S U L T S         |")
print("|                               |")
print(" ------------------------------- ")
print(" Location: ", current_city)
print(" Date:      2023 -",current_month,"-",current_day)

# Print user-specific results

if acc_days < 5:
  print(" Result:    VERY HIGH RISK         ")
  print("                ")
  print(" There are " + str(acc_days) + " days, or " + str(acc_months) + " months, between the \n predicted date of a wildfire and your current date.")
  print(" There may be a wildfire in your area in less than a week.")
elif acc_days >= 5 and acc_days < 15:
  print(" Result:    HIGH RISK         ")
  print("                ")
  print(" There are " + str(acc_days) + " days, or " + str(acc_months) + " months, between the \n predicted date of a wildfire and your current date.")
  print(" There may be a wildfire in your area within a week or 2.")
elif acc_days >=15 and acc_days < 30:
  print(" Result:    MEDIUM RISK         ")
  print("                ")
  print(" There are " + str(acc_days) + " days, or " + str(acc_months) + " months, between the \n predicted date of a wildfire and your current date.")
  print(" There may be a wildfire in your area within a month's time.") 
else:
  print(" Result:    LOW RISK         ")
  print("                ")
  print(" There are " + str(acc_days) + " days, or " + str(acc_months) + " months, between the \n predicted date of a wildfire and your current date.")
  print(" There is minimal risk of a wildfire happening within a month's time.")